Self written AI

Import

In [1]:
import numpy as np
import PIL

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

import os
import csv

In [2]:
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.3, 0.3, 0.3), (0.3,0.3,0.3))
])

In [4]:
class DataProcess():
    def __init__(self, label_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(label_file, header=None)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = PIL.Image.open(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [14]:
class NeuralNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1= nn.Conv2d(3, 30, 20) # (50, 300, 459) ((320-20)/5)+1 = 28
    self.pool= nn.MaxPool2d(4, 4) # (50, 150, 229)
    self.conv2 = nn.Conv2d(30, 60, 60) #(30, 120, 199) => (30, 60, 99.5)
    self.fc1 = nn.Linear(3360, 1600)
    self.fc2 = nn.Linear(1600, 800)
    self.fc3 = nn.Linear(800, 8)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [7]:
class NeuralNetV2(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1= nn.Conv2d(3, 15, 20) # (50, 300, 459) ((320-20)/5)+1 = 28
    self.pool= nn.MaxPool2d(2, 2) # (50, 150, 229)
    self.pool2= nn.MaxPool2d(5, 5) # (50, 150, 229)
    self.conv2 = nn.Conv2d(15, 30, 30) #(30, 120, 199) => (30, 60, 99.5)
    self.conv3 = nn.Conv2d(30, 60, 30) #(60, 30, 69.5) => (60, 15, 35)
    self.fc1 = nn.Linear(5040, 1600)
    self.fc2 = nn.Linear(1600, 800)
    self.fc3 = nn.Linear(800, 8)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool2(F.relu(self.conv3(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [5]:
class NeuralNetV3(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1= nn.Conv2d(3, 15, 20) # (50, 300, 459) ((320-20)/5)+1 = 28
    self.pool= nn.MaxPool2d(2, 2) # (50, 150, 229)
    self.pool2= nn.MaxPool2d(5, 5) # (50, 150, 229)
    self.conv2 = nn.Conv2d(15, 30, 30) #(30, 120, 199) => (30, 60, 99.5)
    self.conv3 = nn.Conv2d(30, 60, 30) #(60, 30, 69.5) => (60, 15, 35)
    self.fc1 = nn.Linear(5040, 2000)
    self.fc2 = nn.Linear(2000, 8)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool2(F.relu(self.conv3(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [6]:
def test(best, name, test_loader):
    test=NeuralNetV3()
    test.to(device)
    test.load_state_dict(torch.load("./models/self_train/"+name+"_Adam_last.pt", weights_only=True))
    test.eval()
    total=0
    correct=0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            images_test, labels_test = data
            images_test, labels_test = images_test.to(device), labels_test.to(device)
            outputs=test(images_test)
            _, predicts = torch.max(outputs, 1)

            total += labels_test.size(0)

            correct += (predicts==labels_test).sum().item()
        
        accuracy = 100*correct/total
        
        if best < accuracy:
            print(f'{best:.4f} < {accuracy: .4f}')
            torch.save(test.state_dict(), "./models/self_train/"+name+"_Adam_best.pt")
        
        print(f'Accuracy: {accuracy}%')

    return accuracy

In [8]:

def train(name, train_loader, net, loss_function, optimizer):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        output = net(inputs)
        # print(output[0].size())
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader): .4f}; {running_loss:.4f};{len(train_loader):.4f}')

    torch.save(net.state_dict(), "./models/self_train/"+name+"_Adam_last.pt")
    return (running_loss / len(train_loader))
   
   
        

In [9]:
def write_csv(name, data):
    with open("./models/self_train/"+name+"_dataset_Adam.csv",'w+',newline='') as file:
        writer=csv.writer(file)
        writer.writerows(data)

In [10]:
FOLDERS=["default","no_bg","random_bg"]

In [11]:
for name in FOLDERS:
    torch.cuda.empty_cache()
    if ('net' in locals() or 'net' in globals()) and ('loss_function' in locals() or 'loss_function' in globals()) and ('optimizer' in locals() or 'optimizer' in globals()):
        del(net, loss_function, optimizer)
    net = NeuralNetV3()
    net.to(device=device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.003)
    # optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)
    csv_data=[['EPOCH','Loss','Accuracy']]
    
    train_data=DataProcess(label_file="./dataset/"+name+"/labels/TRAIN.csv",img_dir="./dataset/"+name+"/TRAIN/",transform=transform)
    test_data=DataProcess(label_file="./dataset/"+name+"/labels/TEST.csv",img_dir="./dataset/"+name+"/TEST/",transform=transform)
    train_loader= torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=0)
    test_loader= torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=False, num_workers=0)
    
    best=0
    EPOCH=60
    
    for epoch in range(EPOCH):
        print(f'Training epoch {epoch}...')

        Loss=train( name, train_loader, net, loss_function, optimizer)

        last=test(best, name, test_loader)
        if best < last:
            best=last
        csv_data.append([epoch,Loss,last])
    
    write_csv(name=name, data=csv_data)

Training epoch 0...
Loss:  354.5712; 18792.2727;53.0000
0.0000 <  26.7267
Accuracy: 26.726726726726728%
Training epoch 1...
Loss:  1.3975; 74.0651;53.0000
Accuracy: 23.843843843843842%
Training epoch 2...
Loss:  1.3918; 73.7631;53.0000
Accuracy: 26.726726726726728%
Training epoch 3...
Loss:  1.3975; 74.0693;53.0000
Accuracy: 26.726726726726728%
Training epoch 4...
Loss:  1.3900; 73.6686;53.0000
Accuracy: 24.924924924924923%
Training epoch 5...
Loss:  1.3948; 73.9270;53.0000
Accuracy: 26.726726726726728%
Training epoch 6...
Loss:  1.3988; 74.1365;53.0000
Accuracy: 26.726726726726728%
Training epoch 7...
Loss:  1.3908; 73.7148;53.0000
Accuracy: 26.726726726726728%
Training epoch 8...
Loss:  1.3929; 73.8237;53.0000
Accuracy: 26.726726726726728%
Training epoch 9...
Loss:  1.3905; 73.6961;53.0000
Accuracy: 24.924924924924923%
Training epoch 10...
Loss:  1.3905; 73.6943;53.0000
Accuracy: 26.726726726726728%
Training epoch 11...
Loss:  1.3899; 73.6670;53.0000
Accuracy: 24.924924924924923%
Tra

In [12]:
validate_defult=DataProcess(label_file="./dataset/default/labels/VALIDATE.csv",img_dir="./dataset/default/VALIDATE/",transform=transform)
validate_defult_loader= torch.utils.data.DataLoader(validate_defult, batch_size=10, shuffle=False, num_workers=0)
validate_no_bg=DataProcess(label_file="./dataset/no_bg/labels/VALIDATE.csv",img_dir="./dataset/no_bg/VALIDATE/",transform=transform)
validate_no_bg_loader= torch.utils.data.DataLoader(validate_no_bg, batch_size=10, shuffle=False, num_workers=0)
validate_random_bg=DataProcess(label_file="./dataset/random_bg/labels/VALIDATE.csv",img_dir="./dataset/random_bg/VALIDATE/",transform=transform)
validate_random_bg_loader= torch.utils.data.DataLoader(validate_random_bg, batch_size=10, shuffle=False, num_workers=0)

validate_loaders=[validate_defult_loader,validate_no_bg_loader,validate_random_bg_loader]

In [23]:
def validate(loader: torch.utils.data.DataLoader):
    validate_model = NeuralNetV3()
    validate_model.to(device)
    validate_model.load_state_dict(torch.load("./models/self_train/default_Adam_best.pt", weights_only=True))
    validate_model.eval()
    total=0
    correct=0
    with torch.no_grad():
        for data in loader:
            image, label =data
            image, label = image.to(device), label.to(device)
            outputs=validate_model(image)
            _, predicts = torch.max(outputs, 1)

            total += label.size(0)

            correct += (predicts==label).sum().item()
        
        accuracy = 100*correct/total
        print(f'Accuracy: {accuracy: .4f}')

In [24]:
for loader in validate_loaders:
    validate(loader)

Accuracy:  24.2442
Accuracy:  23.7530
Accuracy:  24.5057
